In [1]:
! curl "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0012/tlg002/tlg0012.tlg002.perseus-eng3.xml" -o eng.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  850k  100  850k    0     0  1015k      0 --:--:-- --:--:-- --:--:-- 1014k


In [2]:
! curl "https://raw.githubusercontent.com/PerseusDL/canonical-greekLit/master/data/tlg0012/tlg002/tlg0012.tlg002.perseus-grc2.xml" -o grc.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1523k  100 1523k    0     0  1954k      0 --:--:-- --:--:-- --:--:-- 1955k


Update the html files in vim.
```
:%s/"hexameter-line">/"hexameter-line" id="line-">/g 
:let a = 1 | g/class=\"hexameter-line\" id=\"line-\zs\ze\">/ s//\=a/ | let a += 1 
```

First the Greek text:

In [9]:
import xml.etree.ElementTree as ET
tree = ET.parse('grc.xml')
root = tree.getroot()
books = {}
for book in tree.findall(".//*[@subtype='book']"):
    lines = {}
    line_els = book.findall('.//{*}l')
    for l in line_els:
        lines[int(l.attrib["n"])] = "".join(l.itertext()).strip()
    books[int(book.attrib["n"])] = lines
    #print(elem.get("n"))

In [11]:
books[1]

{1: 'ἄνδρα μοι ἔννεπε, μοῦσα, πολύτροπον, ὃς μάλα πολλὰ',
 2: 'πλάγχθη, ἐπεὶ Τροίης ἱερὸν πτολίεθρον ἔπερσεν·',
 3: 'πολλῶν δʼ ἀνθρώπων ἴδεν ἄστεα καὶ νόον ἔγνω,',
 4: 'πολλὰ δʼ ὅ γʼ ἐν πόντῳ πάθεν ἄλγεα ὃν κατὰ θυμόν,',
 5: 'ἀρνύμενος ἥν τε ψυχὴν καὶ νόστον ἑταίρων.',
 6: 'ἀλλʼ οὐδʼ ὣς ἑτάρους ἐρρύσατο, ἱέμενός περ·',
 7: 'αὐτῶν γὰρ σφετέρῃσιν ἀτασθαλίῃσιν ὄλοντο,',
 8: 'νήπιοι, οἳ κατὰ βοῦς Ὑπερίονος Ἠελίοιο',
 9: 'ἤσθιον· αὐτὰρ ὁ τοῖσιν ἀφείλετο νόστιμον ἦμαρ.',
 10: 'τῶν ἁμόθεν γε, θεά, θύγατερ Διός, εἰπὲ καὶ ἡμῖν.',
 11: 'ἔνθʼ ἄλλοι μὲν πάντες, ὅσοι φύγον αἰπὺν\n                  ὄλεθρον,',
 12: 'οἴκοι ἔσαν, πόλεμόν τε πεφευγότες ἠδὲ θάλασσαν·',
 13: 'τὸν δʼ οἶον νόστου κεχρημένον ἠδὲ γυναικὸς',
 14: 'νύμφη πότνιʼ ἔρυκε Καλυψὼ δῖα θεάων',
 15: 'ἐν σπέσσι γλαφυροῖσι, λιλαιομένη πόσιν εἶναι.',
 16: 'ἀλλʼ ὅτε δὴ ἔτος ἦλθε περιπλομένων ἐνιαυτῶν,',
 17: 'τῷ οἱ ἐπεκλώσαντο θεοὶ οἶκόνδε νέεσθαι',
 18: 'εἰς Ἰθάκην, οὐδʼ ἔνθα πεφυγμένος ἦεν ἀέθλων',
 19: 'καὶ μετὰ οἷσι φίλοισι. θεοὶ δʼ ἐλέ

Then the English text:

In [21]:
def parseCard(root, tr_lines = {}, cur = 0, cur_text = ""):
    for item in root:
        if item.text and item.tag != "{http://www.tei-c.org/ns/1.0}note":
            cur_text += item.text
        if item.tag == "{http://www.tei-c.org/ns/1.0}milestone":
            if cur: tr_lines[cur] = cur_text.strip() + ' '
            cur_text = ""
            new_cur = int(item.attrib["n"])
            cur = new_cur if new_cur != 1 else 1
        cur_text, cur, tr_lines = parseCard(item, tr_lines, cur, cur_text)
        if item.tail:
            cur_text += item.tail.strip() + ' '
    return (cur_text, cur, tr_lines)


In [23]:
import xml.etree.ElementTree as ET

tree = ET.parse('eng.xml')
root = tree.getroot()
tr_books = {}
for book in tree.findall(".//*[@subtype='book']"):
    card_els = book.findall(".//*[@subtype='card']")
    tr_lines = {}
    for l in card_els:
        cur = int(l.attrib["n"])
        cur_text, cur, tr_lines = parseCard(l, tr_lines, cur)
        tr_lines[cur] = cur_text.strip()
    tr_books[int(book.attrib["n"])] = tr_lines.copy()
    #print(elem.get("n"))
print(tr_books[1][10])


Of these things, goddess, daughter of Zeus, beginning where thou wilt, tell thou even unto us.
    Now all the rest, as many as had escaped sheer destruction, were at home, safe from both war and sea, but Odysseus alone, filled with longing for his return and for his wife, did the queenly nymph Calypso, that bright goddess, 


In [24]:
import base64
output_dir = "../translation/"

for k, book in books.items():
    tr_book = tr_books[k]

    output_file = open(output_dir + f"translation{k}.js", 'w')
    output_file.write("var translation = {\n")
    
    pi = 0
    lines_in_book = list(book.keys())
    for i in range(1, max(lines_in_book)+1):
        if i not in tr_book:
            output_file.write(f"{i}: {pi},\n")
            continue
        tr = base64.b64encode(bytes(tr_book[i], 'utf-8')).decode("ascii")
        output_file.write(f"{i}: '{tr}',\n")
        pi = i
    output_file.write("}\n")


In [18]:
import re

lines = [l for l in [books[1][str(i)] for i in range(33,68)]]
breaks = [0] + [i+1 for i,l in enumerate(lines) if any(x in ['·',';','.'] for x in l)]
gr_segments = []
for i in range(0, len(breaks)-1):
    f = breaks[i]
    t = breaks[i+1]
    gr_segments += ["\n".join(lines[f:t])]
tr_segments = re.findall('.*?[.\.\?;]', tr_books["1"]["33"])
for i,s in enumerate(gr_segments):
    print(s)
    print('\n')
    print(tr_segments[i])
    print('-----------------------')


KeyError: '33'